Basically our objective is going to be to build an application to figure out where to eat!

# Requirements for the Data Analysis

requests==2.18.4 

API_KEY = YlHRdziZQHwuA-zsqe8cdJd24fSpn6ImQUVnK2ifVExw52A8o6kSnjw2mxtIR-3X0U0fBdVBWDppPv7MWyWr4xoBSeFjBgz9k5yP7S_MMxKTDFWqKOATeUTHRxMjXHYx

pandas==0.20.1

geojsonio==0.0.3

Shapely==1.5.17.post1

CLIENT_ID = dLQC-YFBm9Sjlrw_1z9OTQ

# Imports

<img src="./Yelp_Logo.png" width=500px></img>

These code snippets were modified from the sample code provided by Yelp.

In [2]:
from __future__ import print_function

#import argparse
import json
import pprint
import requests
import sys
import urllib

#Code based off of the sample.py from YELP's API 

try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode
    
API_KEY= "YlHRdziZQHwuA-zsqe8cdJd24fSpn6ImQUVnK2ifVExw52A8o6kSnjw2mxtIR-3X0U0fBdVBWDppPv7MWyWr4xoBSeFjBgz9k5yP7S_MMxKTDFWqKOATeUTHRxMjXHYx"

import pandas as pd
from PIL import Image

In [4]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
REVIEW_PATH = '/v3/businesses/{}/reviews'

# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 10

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location, limit=SEARCH_LIMIT):
    """Query the Search API by a search term and location.
    
    Can add additional terms:
    
        sort_by (string) Optional. Suggestion to the search algorithm that the results be sorted by one of the these
            modes: best_match, rating, review_count or distance. The default is best_match. Note that specifying the
            sort_by is a suggestion (not strictly enforced) to Yelp's search, which considers multiple input parameters
            to return the most relevant results. For example, the rating sort is not strictly sorted by the rating value,
            but by an adjusted rating value that takes into account the number of ratings, similar to a Bayesian average. 
            This is to prevent skewing results to businesses with a single review.
            
        price (string) Optional. Pricing levels to filter the search result with: 1 = $, 2 = $$, 3 = $$$, 4 = $$$$. 
            The price filter can be a list of comma delimited pricing levels. For example, "1, 2, 3" will filter the
            results to show the ones that are $, $$, or $$$.
            
        open_now (boolean) Optional. Default to false. When set to true, only return the businesses open now.
            Notice that open_at and open_now cannot be used together.
    
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': limit
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(API_KEY, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, API_KEY)

def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)
    return response

def query_api_limits(term, location, num_limit = 10):
    """Queries the API by the input values from the user. Will get at most num_limit businesses.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
        num_limit (int): Max number of businesses.
    """
    response = search(API_KEY, term, location, limit=num_limit)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_ids = [i['id'] for i in businesses]
    first_business_id = business_ids[0]
    
    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), first_business_id))
    
    responses = [get_business(API_KEY, business_id) for business_id in business_ids]
    
    return responses

def get_reviews(business_id,api_key=API_KEY,limit=3):
    """Query the Review API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    review_path = REVIEW_PATH.format(business_id)
    
    url_params = {
        'limit': limit
    }
    
    return request(API_HOST,review_path, api_key, url_params=url_params)


## Basic Query

In [5]:
response = query_api("dinner", 'Morningside Heights, NY')

Querying https://api.yelp.com/v3/businesses/search ...
10 businesses found, querying business info for the top result "gJg_Gh2lqrCprDzR6ZP9QQ" ...
Querying https://api.yelp.com/v3/businesses/gJg_Gh2lqrCprDzR6ZP9QQ ...
Result for business "gJg_Gh2lqrCprDzR6ZP9QQ" found:
{ 'alias': 'bar314-new-york',
  'categories': [ {'alias': 'bars', 'title': 'Bars'},
                  {'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
                  {'alias': 'pizza', 'title': 'Pizza'}],
  'coordinates': {'latitude': 40.810854, 'longitude': -73.958384},
  'display_phone': '(646) 682-7645',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': True,
               'open': [ { 'day': 0,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1100'},
                         { 'day': 1,
                           'end': '2200',
                           'is_overnight': False,
                           

## We see we can get the image of the site of question

In [6]:
URL = response['image_url']

with urllib.request.urlopen(URL) as url:
    with open('temp.jpg', 'wb') as f:
        f.write(url.read())

img = Image.open('temp.jpg')

img.show()

# Getting Multiple Responses

In [7]:
responses = query_api_limits("", 'Morningside Heights, NY')

Querying https://api.yelp.com/v3/businesses/search ...
10 businesses found, querying business info for the top result "JV5oa5-KGdiWnqrKPoxSug" ...
Querying https://api.yelp.com/v3/businesses/JV5oa5-KGdiWnqrKPoxSug ...
Querying https://api.yelp.com/v3/businesses/QHYqNQhQ8NVK0RX68Y4PuQ ...
Querying https://api.yelp.com/v3/businesses/8lLs3dsSN-Am2_EtNfbXqA ...
Querying https://api.yelp.com/v3/businesses/vZ5-JXlJS75k8wmPNS5U5w ...
Querying https://api.yelp.com/v3/businesses/8Qe6g3Dv5NXN1Zq-egJk9w ...
Querying https://api.yelp.com/v3/businesses/7tTVuBwJ4LLtQ3-HHW80_A ...
Querying https://api.yelp.com/v3/businesses/E2mNgb479B3BCfwi2G_KdQ ...
Querying https://api.yelp.com/v3/businesses/80fvd_DsoW5XhwQIMBGvtg ...
Querying https://api.yelp.com/v3/businesses/A46G2OAvLxFswiONB50Rrg ...
Querying https://api.yelp.com/v3/businesses/J9xVQScnr0lYWl61_mLXMA ...


In [8]:
def create_new_entry(metadata):
    new_entry = dict()
    new_entry['id']=metadata['id']
    new_entry['name']=metadata['name']
    new_entry['review_count'] =metadata['review_count']
    new_entry['rating']=metadata['rating']
    new_entry['latitude']=metadata['coordinates']['latitude']
    new_entry['longitude']=metadata['coordinates']['longitude']
    new_entry['open']=metadata['hours'][0]['is_open_now']
    return pd.DataFrame(new_entry,index=[0])

def create_category_entry(metadata):
    new_entry = dict()
    categories=dict()
    new_entry['id']=metadata['id']
    for i in range(len(metadata['categories'])):
        categories[i]=metadata['categories'][i]
    new_entry['categories']=categories
    return pd.DataFrame(new_entry)

def createCategoryDataFrame(responses):
    output = pd.concat([create_category_entry(response) for response in responses])
    output = output.reset_index(drop=True)
    return output

def creatingDataFrame(responses):
    output= pd.concat([create_new_entry(response) for response in responses])
    output = output.reset_index(drop=True)
    return output

In [9]:
tempdf = creatingDataFrame(responses)

In [10]:
cat_tempdf = createCategoryDataFrame(responses)

In [11]:
tempdf

,id,name,review_count,rating,latitude,longitude,open
0,JV5oa5-KGdiWnqrKPoxSug,Absolute Bagels,1217,4.5,40.802510,-73.967450,True
1,QHYqNQhQ8NVK0RX68Y4PuQ,Levain Bakery,630,4.5,40.804974,-73.955151,True
2,8lLs3dsSN-Am2_EtNfbXqA,Community Food & Juice,846,3.5,40.805798,-73.965675,True
3,vZ5-JXlJS75k8wmPNS5U5w,Max Soha,385,4.0,40.811302,-73.958183,True
4,8Qe6g3Dv5NXN1Zq-egJk9w,Hungarian Pastry Shop,596,3.5,40.803580,-73.963650,True
5,7tTVuBwJ4LLtQ3-HHW80_A,Riverside Park Conservancy,129,4.5,40.811131,-73.963874,True
6,E2mNgb479B3BCfwi2G_KdQ,Flat Top,403,4.0,40.810041,-73.958693,True
7,80fvd_DsoW5XhwQIMBGvtg,Cathedral Church of St John the Divine,157,4.5,40.803705,-73.963120,True
8,A46G2OAvLxFswiONB50Rrg,Koronet Pizza,589,3.5,40.804420,-73.966056,True
9,J9xVQScnr0lYWl61_mLXMA,Lido,873,4.0,40.804980,-73.954826,True


In [12]:
cat_tempdf

,id,categories
0,JV5oa5-KGdiWnqrKPoxSug,"{'alias': 'bakeries', 'title': 'Bakeries'}"
1,JV5oa5-KGdiWnqrKPoxSug,"{'alias': 'bagels', 'title': 'Bagels'}"
2,QHYqNQhQ8NVK0RX68Y4PuQ,"{'alias': 'bakeries', 'title': 'Bakeries'}"
3,8lLs3dsSN-Am2_EtNfbXqA,"{'alias': 'newamerican', 'title': 'American (N..."
4,8lLs3dsSN-Am2_EtNfbXqA,"{'alias': 'breakfast_brunch', 'title': 'Breakf..."
5,8lLs3dsSN-Am2_EtNfbXqA,"{'alias': 'bars', 'title': 'Bars'}"
6,vZ5-JXlJS75k8wmPNS5U5w,"{'alias': 'italian', 'title': 'Italian'}"
7,8Qe6g3Dv5NXN1Zq-egJk9w,"{'alias': 'bakeries', 'title': 'Bakeries'}"
8,8Qe6g3Dv5NXN1Zq-egJk9w,"{'alias': 'coffee', 'title': 'Coffee & Tea'}"
9,7tTVuBwJ4LLtQ3-HHW80_A,"{'alias': 'parks', 'title': 'Parks'}"


In [13]:
location_ids = [location_id for location_id in cat_tempdf['id'].unique()]

In [14]:
location_ids

['JV5oa5-KGdiWnqrKPoxSug',
 'QHYqNQhQ8NVK0RX68Y4PuQ',
 '8lLs3dsSN-Am2_EtNfbXqA',
 'vZ5-JXlJS75k8wmPNS5U5w',
 '8Qe6g3Dv5NXN1Zq-egJk9w',
 '7tTVuBwJ4LLtQ3-HHW80_A',
 'E2mNgb479B3BCfwi2G_KdQ',
 '80fvd_DsoW5XhwQIMBGvtg',
 'A46G2OAvLxFswiONB50Rrg',
 'J9xVQScnr0lYWl61_mLXMA']

In [15]:
def collate_data(tempdf,location_id):
    if isinstance(location_id, (list,)):
        output=dict()
        
        for loc in location_id:
            aliases = []
            for i in tempdf[tempdf['id']==loc]['categories']:
                aliases.append(i['alias'])
            output[loc]=aliases
        
        return output
    else:
        aliases = []
        for i in tempdf[tempdf['id']==location_id]['categories']:
            aliases.append(i['alias'])
        return {location_id: aliases}

In [16]:
collate_data(cat_tempdf, location_ids)

{'JV5oa5-KGdiWnqrKPoxSug': ['bakeries', 'bagels'],
 'QHYqNQhQ8NVK0RX68Y4PuQ': ['bakeries'],
 '8lLs3dsSN-Am2_EtNfbXqA': ['newamerican', 'breakfast_brunch', 'bars'],
 'vZ5-JXlJS75k8wmPNS5U5w': ['italian'],
 '8Qe6g3Dv5NXN1Zq-egJk9w': ['bakeries', 'coffee'],
 '7tTVuBwJ4LLtQ3-HHW80_A': ['parks'],
 'E2mNgb479B3BCfwi2G_KdQ': ['newamerican', 'cafes', 'breakfast_brunch'],
 '80fvd_DsoW5XhwQIMBGvtg': ['churches', 'landmarks'],
 'A46G2OAvLxFswiONB50Rrg': ['pizza'],
 'J9xVQScnr0lYWl61_mLXMA': ['italian', 'breakfast_brunch', 'cocktailbars']}

In [17]:
get_reviews(location_ids[1])['reviews'][0]['url']

Querying https://api.yelp.com/v3/businesses/QHYqNQhQ8NVK0RX68Y4PuQ/reviews ...


'https://www.yelp.com/biz/levain-bakery-new-york-3?hrid=Necub7tHWFyDhASlJfxTWA&adjust_creative=dLQC-YFBm9Sjlrw_1z9OTQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=dLQC-YFBm9Sjlrw_1z9OTQ'

Huh it seems to only return 3, how are we supposed to do sentiment analysis? This endpoint returns up to three review excerpts for a given business ordered by Yelp's default sort order. This can be edited with the search function.

## Geopandas

In [22]:
import geopandas

ImportError: cannot import name 'urlparse'

In [20]:
from geopandas import GeoDataFrame
from geojsonio import display


class Restaurants(object):
    
    # filename: file with list of bubble tea places and addresses
    def __init__(self, dataframe):
        self.dataframe = dataframe

    # new code here
    def visualizeLocation(self):
        Points = [Point(xy) for xy in zip(self.boba.Longitude, self.boba.Lat)]
        updated = Points
        display(updated.to_json()) 

ImportError: dlopen(/anaconda3/lib/python3.6/site-packages/fiona/ogrext.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libpoppler.71.dylib
  Referenced from: /anaconda3/lib/libgdal.20.dylib
  Reason: image not found

## Sentiment Analysis

In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Bayesian Estimation

## Picking Restaurants with ML

## Location Visualization